In [ ]:
import pandas as pd
from recipe_book import book

table = pd.read_csv('RAW_recipes.csv')
livro = book()
doc_DIR = livro.doc_DIR
data_DIR = livro.data_DIR
text = []
arquivos = []
lista_deNomes = []
arquivosTotal = []

def tokenizeArquivosSepar(arquivos):
    arqTokenizados = []
    for i in range(0,len(arquivos)):
        arquivos[i] = " ".join(arquivos[i])
        arqTokenizados.append(arquivos[i])
        arqTokenizados[i] = arqTokenizados[i].split()
    return arqTokenizados

for l in range(0,len(doc_DIR)):
    op1,op2,op3,op4,op5,op6,op7,op8,op9,op10,op11 = livro.printRecipe(livro.doc_DIR[l])
    arquivos.append([op1,op2,op3,op4,op5,op6,op7,op8,op9,op10,op11])

tokenArq = tokenizeArquivosSepar(arquivos)
print(tokenArq)

In [ ]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import word_tokenize

data = arquivos

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
print(tagged_data)

In [ ]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

In [ ]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])